In [88]:
import tensorflow as tf
import numpy as np

Сначала сгненерируем случайную I/O - таблицу. 

При расчете пропорций потребления конечных потребителей она не учитывается, так что на данном этапе можно генерировать агрегированную.

In [96]:
def get_Z(n,m):
    Z = np.zeros((n+m,n+1))
    Z[:n,:n] = np.random.random((n,n))
    
    S = np.sum(Z[:n,:n], axis = 0)
    lmbds = np.random.random((n,1))
    lmbds = np.array([lmbd/sum(lmbd) for lmbd in lmbds])
    Z[:n,n:n+1] = np.array([S[i] * lmbds[i,:] for i in range(n)])
    
    D = np.sum(Z[:n,:n], axis = 1)
    lmbds = np.random.random((n,m))
    lmbds = np.array([lmbd/sum(lmbd) for lmbd in lmbds])
    Z[n:n+m,:n] = np.array([D[i] * lmbds[i,:] for i in range(n)]).T
    
    return Z

In [138]:
n = tf.constant(3)                # отрасли
m = tf.constant(2)                 # первичные ресурсы
l = tf.constant(6)                 # потребители
Z = tf.Variable(get_Z(n, m))    # I/O - матрица
rho = tf.Variable(np.random.uniform(0.05, 0.95, n),tf.float32) # -1 * параметр CES

Теперь достанем параметры производстьвенных функций из этой I/O таблицы. 

In [142]:
@tf.function
def get_W(n, m, rho, Z):
    A = tf.reduce_sum(Z[:n,:], 1)
    B = tf.math.pow(tf.cast((n + m),tf.float64),(rho-1)/rho)
    W = tf.cast(B * Z[:,:n] / A, tf.float32)
    return W

Имея на руках веса $W$ и параметры $\rho$ можно так же определить функцию, которая по вектору цен на первичные и вторичные ресурсы будет возвращать себестоимость на вторичные ресурсы.  В дальнейшем нужно сделать чтоб они находились рек

In [143]:
print(get_W(n, m, rho, Z))    

tf.Tensor(
[[1.4744526e-04 6.0825015e-04 2.5693888e-07]
 [1.6407868e-04 5.5186573e-04 2.0512373e-06]
 [1.3042946e-05 4.3769600e-04 7.3455061e-07]
 [1.6094932e-04 5.5296061e-04 9.1096359e-07]
 [1.3021755e-04 2.2513377e-03 5.8862923e-07]], shape=(5, 3), dtype=float32)
